In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg

In [2]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)

In [3]:
using Base.Threads;nthreads()

7

In [6]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
dot_num=30
initial_state=kron([1.0 0 0;0 0 0;0 0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3*dim,:)
end


n=kron(eye(3),a₊(dim)*a₋(dim))
en=kron([0 0 0;0 0 0;0 0 1],eye(dim))
rr=kron([0 0 0;0 1 0;0 0 0],eye(dim))
η1=0.15
η2=-0.15
a=a₊(dim)+a₋(dim)
ag2=kron([0 0 0;0 0 0;1 0 0],exp(im*η1*a))
ag3=kron([0 0 1;0 0 0;0 0 0],exp(-im*η1*a))
ag4=ag2+ag3
ar2=kron([0 0 0;0 0 0;0 1 0],exp(im*η2*a))
ar3=kron([0 0 0;0 0 1;0 0 0],exp(-im*η2*a))
ar4=ar2+ar3

E=eye(3*dim)
γ1=20/3*ν
γ2=40/3*ν

function diss1()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η1*theta*a)
        aup=kron([0 0 0;0 0 0;1 0 0],jump')
        adown=kron([0 0 1;0 0 0;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η2*theta*a)
        aup=kron([0 0 0;0 0 0;0 1 0],jump')
        adown=kron([0 0 0;0 0 1;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

dissipation1=γ1*diss1()
dissipation2=γ2*diss2()

# Δg=(Ωr^2+Ωg^2)/ν-ν
# Δr=Δg

function eit_Lindbladian_vec(Ωg,Ωr,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

instate=dm2vec(initial_state);


In [41]:
function sweep_detuning_100()
    ttot=100
    initial_x=[5.121757941 10.35114471]
    sweep_num=140
    start=10
    stop=150
    d=[]
    somega_g=[]
    somega_r=[]
    sn=[]
    for i in 1:sweep_num
        println(" ")
        println(i," ")
        m=start+(i-1)*(stop-start)/sweep_num
        function feval(x)
            f=exp(ttot*eit_Lindbladian_vec(x[1],x[2],m))*instate
            nbose=real(tr(n*vec2dm(f)))
            print(" f ",nbose," ")
            return nbose
        end        

        a=optimize(feval,initial_x,LBFGS())
        initial_x=Optim.minimizer(a) 
        append!(somega_g,initial_x[1])  
        append!(somega_r,initial_x[2])
        append!(sn,Optim.minimum(a)[1])
        append!(d,m)
        df1=DataFrame(delta=d,omega_g=somega_g,omega_r=somega_r,n=sn);CSV.write("eit_sweep_detuning_100.csv", df1);
    end
    return somega_g,somega_r,sn,d
end
@time res=sweep_detuning_100()


1 
 f 0.24480412738240356 f 0.2448064942559322

LoadError: [91mInterruptException:[39m